<a href="https://colab.research.google.com/github/hublun/Bayesian_Aggregation_Average_Data/blob/master/ODR_2024_1_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [ODE Based Models in Stan](https://mc-stan.org/events/stancon2017-notebooks/stancon2017-margossian-gillespie-ode.html)

#### [Example: simple harmonic oscillator](https://mc-stan.org/docs/stan-users-guide/example-simple-harmonic-oscillator.html#example-simple-harmonic-oscillator)

#### [Bayesian workflow for disease transmission modeling in Stan](https://mc-stan.org/users/documentation/case-studies/boarding_school_case_study.html)

#### [ Ordinary differential equation (ODE) solvers](https://mc-stan.org/docs/functions-reference/functions-ode-solver.html#functions-ode-solver)


In [1]:
install.packages("rstan", repos = c("https://mc-stan.org/r-packages/", getOption("repos")))

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘matrixStats’, ‘StanHeaders’, ‘inline’, ‘gridExtra’, ‘Rcpp’, ‘RcppParallel’, ‘loo’, ‘QuickJSR’, ‘RcppEigen’, ‘BH’




In [5]:
library(rstan)

In [2]:
parallel::detectCores()

[1] 8

In [4]:
#example(stan_model, package = "rstan", run.dontrun = TRUE)

In [9]:
install.packages('tidybayes')

Installing package into ‘/home/dulunche/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)

also installing the dependencies ‘svUnit’, ‘ggdist’, ‘coda’, ‘arrayhelpers’






---



## COVID-19 transmission in Switzerland

#### To illustrate the ideas presented in this tutorial, we are going to tackle a more complex example: COVID-19 transmission in Switzerland during the period from February 2020 to June 2020. We’ll see that in this more complicated setting, sampling from the posterior distribution can be difficult, and we’ll need to rely on Stan’s diagnostics to check that our inference is reliable.

In [13]:
library(tidybayes)
library(tidyverse)
library(gridExtra)
df_swiss <- read_csv("./disease_transmission_workflow/data/swiss_agg_data.csv")

Rows: 132 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (8): onset_dt, test_dt, report_dt, hospit_dt, death_dt, death_dt_hospit...
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [14]:
df_swiss

date,onset_dt,test_dt,report_dt,hospit_dt,death_dt,death_dt_hospit,death_dt_nohospit,total_tested
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020-01-30,1,0,0,1,0,0,0,40
2020-01-31,0,0,0,2,0,0,0,32
2020-02-01,3,0,0,2,0,0,0,19
2020-02-02,1,0,0,0,0,0,0,11
2020-02-03,1,0,0,2,0,0,0,14
2020-02-04,2,0,0,2,0,0,0,26
2020-02-05,0,0,0,1,0,0,0,28
2020-02-06,0,0,0,1,0,0,0,13
2020-02-07,2,0,0,1,0,0,0,15
